# Important Library

We use the following libraries in this project:
* [Pandas](https://pandas.pydata.org/) - Data Analysis
* [Numpy](https://numpy.org/) - Data Analysis
* [Matplotlib](https://matplotlib.org/) - Data Visualization
* [Seaborn](https://seaborn.pydata.org/) - Data Visualization
* [Scikit-learn](https://scikit-learn.org/stable/) - Machine Learning
* [Tensorflow](https://www.tensorflow.org/) - Machine Learning
* [Keras](https://keras.io/) - Machine Learning

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

print(tf.__version__)
print(np.__version__)
print(sns.__version__)

# Exploring the Dataset

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    directory='./data/Vegetable Images/train',
    seed = 42,
    image_size=(224, 224),
    batch_size=64,
    shuffle=True,
)

test_data = tf.keras.utils.image_dataset_from_directory(
    directory='./data/Vegetable Images/test',
    seed = 42,
    image_size=(224, 224),
    batch_size=64,
    shuffle=False,
)

validation_data = tf.keras.utils.image_dataset_from_directory(
    directory='./data/Vegetable Images/validation',
    seed = 42,
    image_size=(224, 224),
    batch_size=64,
    shuffle=True,
)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


# Creating Small Model

In [ ]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True,
    start_from_epoch=0
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1,
    min_lr=1e-7,
    min_delta=0.0001,
    mode='auto'
)

In [ ]:
# MobileNetV2 Pretrained Model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    classifier_activation='softmax'
)

In [ ]:
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(15, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              metrics=['accuracy'])

# Training the Model

In [ ]:
# tensorboard
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

history = model.fit(train_data,
                    epochs=7, 
                    validation_data=validation_data,
                    verbose=1,
                    callbacks=[early_stopping, 
                               reduce_lr, 
                               tensorboard_callback
                              ]
                    )


Epoch 1/10
235/235 [==============================] - 2299s 10s/step - loss: 0.2839 - accuracy: 0.9299 - val_loss: 0.6803 - val_accuracy: 0.7863 - lr: 1.0000e-04
Epoch 2/10
 45/235 [====>.........................] - ETA: 30:31 - loss: 0.0176 - accuracy: 0.9958

KeyboardInterrupt: 

# Model Evaluation

In [ ]:
# Save the model
model.save('vegetable_classification_model.h5')

# Converting the Model to TFLite

In [ ]:
# Load the model
model = tf.keras.models.load_model('vegetable_classification_model.h5')

# Quantize and convert to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized model
with open('vegetable_classification_model_quantized.tflite', 'wb') as f:
    f.write(quantized_tflite_model)


INFO:tensorflow:Assets written to: C:\Users\brian\AppData\Local\Temp\tmp7mwya219\assets


INFO:tensorflow:Assets written to: C:\Users\brian\AppData\Local\Temp\tmp7mwya219\assets
